# BeeAI ReAct agents

The BeeAI ReAct agent is a pre-canned ReAct agent implementation that can be configured with tools and instructions. 

The ReAct pattern, short for Reasoning and Acting, is a framework used in AI models, particularly in language models, that separates the reasoning process from the action-taking process. 
It enhances the model's ability to handle complex queries by allowing it to reason about the problem, decide on an action, and then observe the result of that action to inform further reasoning and actions. 

The ReAct agent provides a convenient out of the box agent implementation.

## Basic ReAct Agent

To configure a ReAct agent you will need to define a ChatModel and construct a BeeAgent.

In this example we do not provide any tools to the agent, it will attempt to provide an answer from its own memory.

Try changing the text input in the call to `agent.run()` to experiment with getting different answers.

In [ ]:
import warnings

from beeai_framework.agents.bee.agent import BeeAgent
from beeai_framework.agents.types import BeeInput, BeeRunInput, BeeRunOutput
from beeai_framework.backend.chat import ChatModel
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory

warnings.simplefilter("ignore", UserWarning)

# Construct ChatModel
chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")

# Construct Agent instance with the chat model
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[], memory=UnconstrainedMemory()))

# Run the agent
result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What chemical elements make up a water molecule?"))

print(result.result.text)

## Using Tools

To move beyond chatting to an LLM, tools can be provided to the agent.  There are different ways to do this via built-in tools from the framework, importing tools from other libraries or writing your own custom tooling.

## Built-in tools

BeeAI comes with some built-in tools that are provided as part of the library.  These can easily be imported and added to the agent.

Your agent is now capable of doing a little more than the underlying Large Language Model is able to do by itself as it can now rely on tools to fetch additional knowledge/context from elsewhere.

In this example we give the agent a weather forecast lookup tool called OpenMeteoTool.

In [ ]:
from beeai_framework.backend.chat import ChatModel
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool

chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")

# create an agent using the default LLM and add the OpenMeteoTool that is capable of fetching weather-based information
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[OpenMeteoTool()], memory=UnconstrainedMemory()))

result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What's the current weather in London?"))

print(result.result.text)

### Custom Tools

Tools can be written from scratch and added to your agent.

Use the `@tool` decorator on your tool function and pass it into the agent in the same way as built-int tools.

In [ ]:
import json
from urllib.parse import quote

import requests

from beeai_framework import BeeAgent, tool
from beeai_framework.backend.chat import ChatModel
from beeai_framework.tools.tool import StringToolOutput
from beeai_framework.tools.weather.openmeteo import OpenMeteoTool


# defining a tool using the `tool` decorator
# Note: the pydoc is important as it serves as the tool description to the agent
@tool
def basic_calculator(expression: str) -> str:
    """
    A calculator tool that performs mathematical operations.

    Args:
        expression: The mathematical expression to evaluate (e.g., "2 + 3 * 4").

    Returns:
        The result of the mathematical expression
    """
    try:
        encoded_expression = quote(expression)
        math_url = f"https://newton.vercel.app/api/v2/simplify/{encoded_expression}"

        response = requests.get(
            math_url,
            headers={"Accept": "application/json"},
        )
        response.raise_for_status()
        return StringToolOutput(json.dumps(response.json()))

    except Exception as e:
        raise RuntimeError(f"Error evaluating expression: {e!s}") from Exception


chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[basic_calculator], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(run_input=BeeRunInput(prompt="What is the square root of 36?"))
print(result.result.text)

### Imported Tools

Tools can be imported from other libraries.

The example below shows how to integrate a tool from LangChain.  It also demonstrates long form tool writing without the use of the `@tool` decorator (see below for how this can be re-written in short form using the `@tool` decorator).

In [ ]:
from typing import Any

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field

from beeai_framework import BeeAgent
from beeai_framework.tools import Tool


class LangChainWikipediaToolInput(BaseModel):
    query: str = Field(description="The topic or question to search for on Wikipedia.")


class LangChainWikipediaTool(Tool):
    """Adapter class to integrate LangChain's Wikipedia tool with our framework"""

    name = "Wikipedia"
    description = "Search factual and historical information from Wikipedia about given topics."
    input_schema = LangChainWikipediaToolInput

    def __init__(self) -> None:
        super().__init__()
        wikipedia = WikipediaAPIWrapper()
        self.wikipedia = WikipediaQueryRun(api_wrapper=wikipedia)

    def _run(self, input: LangChainWikipediaToolInput, _: Any | None = None) -> None:
        query = input.query
        try:
            result = self.wikipedia.run(query)
            return StringToolOutput(result=result)
        except Exception as e:
            print(f"Wikipedia search error: {e!s}")
            return f"Error searching Wikipedia: {e!s}"


chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[LangChainWikipediaTool()], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(
    run_input=BeeRunInput(prompt="Who is the current president of the European Commission?")
)
print(result.result.text)

The above example can be re-written in shorter form by adding the `@tool` decorator.

In [ ]:
from langchain_community.tools import WikipediaQueryRun  # noqa: F811
from langchain_community.utilities import WikipediaAPIWrapper  # noqa: F811

from beeai_framework import BeeAgent, tool


# defining a tool using the `tool` decorator
# Note: the pydoc is important as it serves as the tool description to the agent
@tool
def langchain_wikipedia_tool(expression: str) -> str:
    """
    Search factual and historical information, including biography, history, politics, geography, society, culture,
    science, technology, people, animal species, mathematics, and other subjects.

    Args:
        expression: The topic or question to search for on Wikipedia.

    Returns:
        The information found via searching Wikipedia.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return StringToolOutput(wikipedia.run(expression))


# using the tool in an agent
chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:8b")
agent = BeeAgent(bee_input=BeeInput(llm=chat_model, tools=[langchain_wikipedia_tool], memory=UnconstrainedMemory()))
result: BeeRunOutput = await agent.run(
    run_input=BeeRunInput(prompt="Who is the current president of the European Commission?")
)
print(result.result.text)